# Set up

In [32]:
import os
import pickle
import json
from loguru import logger
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time

import mlflow

In [2]:
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [4]:
TESTING = False
DEBUG = True
LOG_TO_MLFLOW = True

In [5]:
import logging
import sys

if DEBUG:
    logging.getLogger('llama_index').addHandler(logging.StreamHandler(stream=sys.stdout))
    logging.getLogger('llama_index').setLevel(logging.DEBUG)

In [6]:
if LOG_TO_MLFLOW:
    RUN_NAME = "exp_001_qdrant_togetherai_llama3"
    RUN_DESCRIPTION = """
# Qdrant with TogetherAI Llama3 model
"""
    mlflow.set_experiment("Chain Frost - LlamaIndex Blog QnA Chatbot")
    mlflow.start_run(run_name=RUN_NAME, description=RUN_DESCRIPTION)
    mlflow.log_param("TESTING", TESTING)

In [7]:
NOTEBOOK_CACHE_DP = f'data/001/{RUN_NAME}'
os.makedirs(NOTEBOOK_CACHE_DP, exist_ok=True)

if LOG_TO_MLFLOW:
    mlflow.log_param("NOTEBOOK_CACHE_DP", NOTEBOOK_CACHE_DP)

# Load data

In [8]:
DATA_FP = '../crawl_llamaindex_blog/data/blogs.json'
with open(DATA_FP, 'r') as f:
    data = json.load(f)

In [9]:
len(data)

159

In [10]:
data[:5]

[{'title': 'Automate online tasks with MultiOn and LlamaIndex',
  'content': 'Introduction MultiOn is an AI agents platform designed to facilitate the autonomous completion of tasks in any web environment. It empowers developers to build AI agents that can manage online activities from start to finish, handling everything from simple data retrieval to complex interactions. LlamaIndex complements this by providing an orchestration framework that bridges the gap between private and public data essential for building applications with Large Language Models. It facilitates data ingestion, indexing, and querying, making it indispensable for developers looking to leverage generative AI. In this article, we\'ll demonstrate how MultiOn\'s capabilities can be seamlessly integrated within the LlamaIndex framework, showcasing a practical application that leverages both technologies to automate and streamline web interactions. Technical walkthrough: Integrating MultiOn with LlamaIndex Let’s explor

# Check data

In [11]:
data[0]['content']

'Introduction MultiOn is an AI agents platform designed to facilitate the autonomous completion of tasks in any web environment. It empowers developers to build AI agents that can manage online activities from start to finish, handling everything from simple data retrieval to complex interactions. LlamaIndex complements this by providing an orchestration framework that bridges the gap between private and public data essential for building applications with Large Language Models. It facilitates data ingestion, indexing, and querying, making it indispensable for developers looking to leverage generative AI. In this article, we\'ll demonstrate how MultiOn\'s capabilities can be seamlessly integrated within the LlamaIndex framework, showcasing a practical application that leverages both technologies to automate and streamline web interactions. Technical walkthrough: Integrating MultiOn with LlamaIndex Let’s explore a practical example where MultiOn and LlamaIndex work in tandem to manage e

# Prepare documents

In [12]:
input_data = data
if TESTING:
    input_data = data[:2]
logger.info(f"{len(input_data)=}")
if LOG_TO_MLFLOW:
    mlflow.log_param("len_input_data", len(input_data))

2024-07-23 13:29:54.135 | INFO     | __main__:<module>:4 - len(input_data)=159


In [13]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

documents = []
for record in input_data:
    title = record['title']
    metadata = {
        'title': title,
        'author': record['author'],
        'date': record['date'],
        'tags': ', '.join(record['tags'])
    }
    text = f"{title}\n{record['content']}"
    doc = Document(text=text, metadata=metadata)
    documents.append(doc)

In [14]:
documents[0]

Document(id_='0509b067-43b1-47e0-8a92-457d9f25b80b', embedding=None, metadata={'title': 'Automate online tasks with MultiOn and LlamaIndex', 'author': 'MultiOn', 'date': 'May 23, 2024', 'tags': 'automation, Agents'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Automate online tasks with MultiOn and LlamaIndex\nIntroduction MultiOn is an AI agents platform designed to facilitate the autonomous completion of tasks in any web environment. It empowers developers to build AI agents that can manage online activities from start to finish, handling everything from simple data retrieval to complex interactions. LlamaIndex complements this by providing an orchestration framework that bridges the gap between private and public data essential for building applications with Large Language Models. It facilitates data ingestion, indexing, and querying, making it indispensable for developers looking to leverage generative AI. In this article, we\'ll demonstr

In [15]:
documents[1].metadata

{'title': 'Simplify your RAG application architecture with LlamaIndex + PostgresML',
 'author': 'PostgresML',
 'date': 'May 28, 2024',
 'tags': 'Managed Indexes'}

In [16]:
if LOG_TO_MLFLOW:
    mlflow.log_param("len_documents", len(documents))

## Setting LLM

In [17]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, ServiceContext

In [18]:
# LLM_OPTION = 'openai'
# LLM_OPTION = 'ollama'
LLM_OPTION = 'togetherai'

# LLM_MODEL_NAME = 'llama3'
# LLM_MODEL_NAME = 'gpt-3.5-turbo'
LLM_MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct-Lite'

# EMBED_OPTION = 'openai'
# EMBED_OPTION = 'togetherai'
# EMBED_OPTION = 'ollama'
EMBED_OPTION = 'huggingface'

# EMBED_MODEL_NAME = 'llama3'
# EMBED_MODEL_NAME = 'togethercomputer/m2-bert-80M-2k-retrieval'
EMBED_MODEL_NAME = "BAAI/bge-small-en-v1.5"

if LOG_TO_MLFLOW:
    mlflow.log_param("LLM_OPTION", LLM_OPTION)
    mlflow.log_param("LLM_MODEL_NAME", LLM_MODEL_NAME)
    mlflow.log_param("EMBED_OPTION", EMBED_OPTION)
    mlflow.log_param("EMBED_MODEL_NAME", EMBED_MODEL_NAME)

In [21]:
# LLM options
if LLM_OPTION == 'ollama':
    LLM_SERVER_HOST = '192.168.100.14'
    LLM_SERVER_PORT = 11434
    base_url = f'http://{LLM_SERVER_HOST}:{LLM_SERVER_PORT}'
    llm = Ollama(base_url=base_url, model=LLM_MODEL_NAME, request_timeout=60.0)
    !ping -c 1 $LLM_SERVER_HOST
elif LLM_OPTION == 'openai':
    from llama_index.llms.openai import OpenAI
    llm = OpenAI(model=LLM_MODEL_NAME)
elif LLM_OPTION == 'togetherai':
    from llama_index.llms.together import TogetherLLM
    llm = TogetherLLM(model=LLM_MODEL_NAME)

# Embed options
if EMBED_OPTION == 'huggingface':
    from llama_index.embeddings.huggingface import HuggingFaceEmbedding
    embed_model = HuggingFaceEmbedding(
        model_name=EMBED_MODEL_NAME
    )
elif EMBED_OPTION == 'openai':
    from llama_index.embeddings.openai import OpenAIEmbedding
    embed_model = OpenAIEmbedding()
elif EMBED_OPTION == 'togetherai':
    from llama_index.embeddings.together import TogetherEmbedding
    embed_model = TogetherEmbedding(EMBED_MODEL_NAME)
elif EMBED_OPTION == 'ollama':
    from llama_index.embeddings.ollama import OllamaEmbedding
    embed_model = OllamaEmbedding(
        model_name=EMBED_MODEL_NAME,
        base_url=base_url,
        ollama_additional_kwargs={"mirostat": 0},
    )

logger.info(f"LLM:\n{repr(llm)}")
logger.info(f"Embed model:\n{repr(embed_model)}")

2024-07-23 13:30:32.212 | INFO     | __main__:<module>:35 - LLM:
TogetherLLM(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x793a452ab550>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x793afa355080>, completion_to_prompt=<function default_completion_to_prompt at 0x793afa3c3420>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='meta-llama/Meta-Llama-3-8B-Instruct-Lite', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='3cf613093b6eb9b479c341126dc8d3761c67f9340d0a4a8e1fdc62ed41b58126', api_base='https://api.together.xyz/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=False, tokenizer=None)
2024-07-23 13:30:32.213 | INFO     | __main__:<module>:36 - Embed model:
HuggingFaceEmbedding(model_name='BAAI/bge-small-e

In [22]:
embed_model_dim = len(embed_model.get_text_embedding('sample text to find embedding dimensions'))
Settings.embed_model = embed_model
Settings.llm = llm

if LOG_TO_MLFLOW:
    mlflow.log_param("embedding_model_dim", embed_model_dim)
    mlflow.log_param("LLM_MODEL", repr(llm))
    mlflow.log_param("EMBEDDING_MODEL", repr(embed_model))

# Index embeddings

## Qdrant as VectorStore

In [23]:
import qdrant_client
from qdrant_client.models import Distance, VectorParams
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [25]:
import string

def substitute_punctuation(text):
    # Create a translation table that maps each punctuation character to an underscore
    translator = str.maketrans(string.punctuation, '_' * len(string.punctuation))
    # Translate the text using the translation table
    return text.translate(translator)

collection_raw_name = f"{EMBED_OPTION}__{EMBED_MODEL_NAME}"
logger.info(f"{substitute_punctuation(collection_raw_name)=}")

2024-07-23 13:30:40.391 | INFO     | __main__:<module>:10 - substitute_punctuation(collection_raw_name)='huggingface__BAAI_bge_small_en_v1_5'


In [28]:
RECREATE_INDEX = True

COLLECTION = substitute_punctuation(collection_raw_name)
NODES_PERSIST_FP = f'{NOTEBOOK_CACHE_DP}/nodes.pkl'

if LOG_TO_MLFLOW:
    mlflow.log_param(f"COLLECTION", COLLECTION)

In [29]:
qdrantdb = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://<host>:<port>"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="<qdrant-api-key>",
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://<host>:<port>"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="<qdrant-api-key>",
)
collection_exists = qdrantdb.collection_exists(COLLECTION)
if RECREATE_INDEX or not collection_exists:
    if collection_exists:
        logger.info(f"Deleting existing Qdrant collection...")
        qdrantdb.delete_collection(COLLECTION)
    if os.path.exists(NODES_PERSIST_FP):
        logger.info(f"Deleting persisted nodes object at {NODES_PERSIST_FP}...")
        os.remove(NODES_PERSIST_FP)
    logger.info(f"Creating new Qdrant collection...")
    qdrantdb.create_collection(
        COLLECTION,
        vectors_config=VectorParams(size=embed_model_dim, distance=Distance.COSINE),
    )
else:
    logger.info(f"Use existing Qdrant collection")
db_collection = qdrantdb.get_collection(COLLECTION)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=COLLECTION,
    aclient=aqdrantdb,
    prefer_grpc=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

2024-07-23 13:30:58.725 | INFO     | __main__:<module>:35 - Creating new Qdrant collection...


Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [30]:
CHUNKER = "SentenceSplitter"
CHUNKER_CONFIG = {
    "chunk_size": 512,
    "chunk_overlap": 10
}
if LOG_TO_MLFLOW:
    mlflow.log_param("CHUNKER", CHUNKER)
    for k, v in CHUNKER_CONFIG.items():
        mlflow.log_param(f"CHUNKER__{k}", v)

In [33]:
t0 = time.perf_counter()
db_collection_count = db_collection.indexed_vectors_count

if db_collection_count > 0 and RECREATE_INDEX == False:
    logger.info(f"Loading index from existing DB...")
    with open(NODES_PERSIST_FP, 'rb') as f:
        nodes = pickle.load(f)
else:
    logger.info(f"Creating new DB index...")
    # Generate nodes
    # https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/
    
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache
    
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(**CHUNKER_CONFIG),
            TitleExtractor(),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = os.cpu_count() - 1
    logger.info(f"Running Ingestion Pipeline with {num_workers=}")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers)
    with open(NODES_PERSIST_FP, 'wb') as f:
        pickle.dump(nodes, f)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
t1 = time.perf_counter()
logger.info(f"Indexing {len(documents)} into VectorStoreIndex took {t1 - t0:,.0f}s")

2024-07-23 13:32:17.510 | INFO     | __main__:<module>:9 - Creating new DB index...


> Adding chunk: Automate online tasks with MultiOn and LlamaInd...
> Adding chunk: All parameters are required
        
        If...
> Adding chunk: print (agent.chat( "browse to the latest email ...
> Adding chunk: The email was authenticated and passed SPF and ...
> Adding chunk: As these technologies evolve, they will continu...
> Adding chunk: Simplify your RAG application architecture with...
> Adding chunk: On the PostgresML cloud, you can perform vector...
> Adding chunk: Step 2: Create the PostgresML Managed Index Fir...
> Adding chunk: The PostgresML Managed Index is doing embedding...
> Adding chunk: LlamaIndex Newsletter 2024-06-04
Hello, LlamaIn...
> Adding chunk: Blogpost ,  Tweet . We have integrated with Mil...
> Adding chunk: Memary is a fully open-source reference impleme...
> Adding chunk: Batch inference with MyMagic AI and LlamaIndex
...
> Adding chunk: import  asyncio
 from  llama_index.llms.mymagic...
> Adding chunk: The user queries will be aggregated and append

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.17s/it]
2024-07-23 13:52:04.945 | INFO     | __main__:<module>:34 - Indexing 159 into VectorStoreIndex took 1,187s


In [34]:
logger.info(f"Indexed {len(nodes)} nodes into Vector Store")
if LOG_TO_MLFLOW:
    mlflow.log_param("len_nodes", len(nodes))

2024-07-23 13:52:21.772 | INFO     | __main__:<module>:1 - Indexed 808 nodes into Vector Store


In [23]:
import os
import pickle
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [24]:
RECREATE_INDEX = False

COLLECTION = 'togetherai'
NOTEBOOK_CACHE_DP = 'data/001/togetherai'
NODES_PERSIST_FP = f'{NOTEBOOK_CACHE_DP}/nodes.pkl'
os.makedirs(NOTEBOOK_CACHE_DP, exist_ok=True)

In [25]:
db = chromadb.PersistentClient(path=f"{NOTEBOOK_CACHE_DP}/chroma_db")
collection_exists = COLLECTION in [c.name for c in db.list_collections()]
if RECREATE_INDEX or not collection_exists:
    logger.info(f"Creating new ChromaDB collection...")
    if collection_exists:
        logger.info(f"Deleting existing ChromaDB collection...")
        db.delete_collection(COLLECTION)
    if os.path.exists(NODES_PERSIST_FP):
        logger.info(f"Deleting persisted nodes object at {NODES_PERSIST_FP}...")
        os.remove(NODES_PERSIST_FP)
else:
    logger.info(f"Use existing ChromaDB collection")
chroma_collection = db.get_or_create_collection(COLLECTION)

2024-07-23 12:09:50.358 | INFO     | __main__:<module>:12 - Use existing ChromaDB collection


In [26]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [27]:
CHUNKER = "SentenceSplitter"
CHUNKER_CONFIG = {
    "chunk_size": 512,
    "chunk_overlap": 10
}
if LOG_TO_MLFLOW:
    mlflow.log_param("CHUNKER", CHUNKER)
    for k, v in CHUNKER_CONFIG.items():
        mlflow.log_param(f"CHUNKER__{k}", v)

In [28]:
if chroma_collection.count() > 0 and RECREATE_INDEX == False:
    logger.info(f"Loading index from existing ChromaDB...")
    with open(NODES_PERSIST_FP, 'rb') as f:
        nodes = pickle.load(f)
else:
    logger.info(f"Creating new ChromaDB index...")
    # Generate nodes
    # https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/
    
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache
    
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(**CHUNKER_CONFIG),
            TitleExtractor(),
            embedding,
        ],
        vector_store = vector_store
    )
    
    # Need to use await and arun here to run the pipeline else error
    # Ref: https://docs.llamaindex.ai/en/stable/examples/ingestion/async_ingestion_pipeline/
    # Ref: https://github.com/run-llama/llama_index/issues/13904#issuecomment-2145561710
    nodes = await pipeline.arun(documents=documents)
    with open(NODES_PERSIST_FP, 'wb') as f:
        pickle.dump(nodes, f)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

2024-07-23 12:09:51.484 | INFO     | __main__:<module>:2 - Loading index from existing ChromaDB...


#### Inspect nodes

# Query engine

In [35]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [36]:
RETRIEVAL_TOP_K = 2
# Need to be able to control this cutoff until specify it
RETRIEVAL_SIMILARITY_CUTOFF = None
# RETRIEVAL_SIMILARITY_CUTOFF = 0.3

if LOG_TO_MLFLOW:
    mlflow.log_param("RETRIEVAL_TOP_K", RETRIEVAL_TOP_K)
    if RETRIEVAL_SIMILARITY_CUTOFF:
        mlflow.log_param("RETRIEVAL_SIMILARITY_CUTOFF", RETRIEVAL_SIMILARITY_CUTOFF)

In [37]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=RETRIEVAL_TOP_K,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

node_postprocessors = []

if RETRIEVAL_SIMILARITY_CUTOFF is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=RETRIEVAL_SIMILARITY_CUTOFF))

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

In [38]:
question = "What is MultiOn?"
response = query_engine.query(question)
logger.info(response)

> Top 2 nodes:
> [Node f0812544-41a5-4374-9c28-4a89704c920f] [Similarity score:             0.708697] Automate online tasks with MultiOn and LlamaIndex
Introduction MultiOn is an AI agents platform d...
> [Node dbc2d800-7fe5-4d49-81cf-a6f5f6d5b1ce] [Similarity score:             0.691203] The email was authenticated and passed SPF and DKIM checks.

In response to the last email, I wou...


2024-07-23 13:53:04.124 | INFO     | __main__:<module>:3 - MultiOn is an AI agents platform designed to facilitate the autonomous completion of tasks in any web environment. It empowers developers to build AI agents that can manage online activities from start to finish, handling everything from simple data retrieval to complex interactions.


# Evaluation

## Retrieval Evaluation

### Building synthetic evaluation dataset

In [39]:
with open(NODES_PERSIST_FP, 'rb') as f:
    nodes = pickle.load(f)

In [40]:
from llama_index.core.evaluation import generate_question_context_pairs, EmbeddingQAFinetuneDataset

In [41]:
RECREATE_RETRIEVAL_EVAL_DATASET = True
RETRIEVAL_EVAL_DATASET_FP = f"{NOTEBOOK_CACHE_DP}/llamaindex_blog_retrieval_eval_dataset.json"
RETRIEVAL_NUM_SAMPLE_NODES = 10
RETRIEVAL_NUM_SAMPLE_NODES = min(len(nodes), RETRIEVAL_NUM_SAMPLE_NODES)
RETRIEVAL_NUM_QUESTIONS_PER_CHUNK = 2
if LOG_TO_MLFLOW:
    mlflow.log_param("RETRIEVAL_NUM_QUESTIONS_PER_CHUNK", RETRIEVAL_NUM_QUESTIONS_PER_CHUNK)
    mlflow.log_param("RETRIEVAL_NUM_SAMPLE_NODES", RETRIEVAL_NUM_SAMPLE_NODES)

In [42]:
if RECREATE_RETRIEVAL_EVAL_DATASET or not os.path.exists(RETRIEVAL_EVAL_DATASET_FP):
    if RETRIEVAL_NUM_SAMPLE_NODES:
        logger.info(f"Sampling {RETRIEVAL_NUM_SAMPLE_NODES} nodes for retrieval evaluation...")
        np.random.seed(41)
        retrieval_eval_nodes = np.random.choice(nodes, RETRIEVAL_NUM_SAMPLE_NODES)
    else:
        logger.info(f"Using all nodes for retrieval evaluation")
        retrieval_eval_nodes = nodes
else:
    logger.info(f"Loading retrieval_eval_nodes from {RETRIEVAL_EVAL_DATASET_FP}...")
    with open(RETRIEVAL_EVAL_DATASET_FP, 'r') as f:
        retrieval_eval_nodes = json.load(f)

2024-07-23 13:53:26.990 | INFO     | __main__:<module>:3 - Sampling 10 nodes for retrieval evaluation...


In [43]:
if RECREATE_RETRIEVAL_EVAL_DATASET or not os.path.exists(RETRIEVAL_EVAL_DATASET_FP):
    logger.info(f"Creating new synthetic retrieval eval dataset...")
    retrieval_eval_dataset = generate_question_context_pairs(
        retrieval_eval_nodes, llm=llm, num_questions_per_chunk=RETRIEVAL_NUM_QUESTIONS_PER_CHUNK
    )
    retrieval_eval_dataset.save_json(RETRIEVAL_EVAL_DATASET_FP)
else:
    logger.info(f"Loading existing synthetic retrieval eval dataset at {RETRIEVAL_EVAL_DATASET_FP}...")
    retrieval_eval_dataset = EmbeddingQAFinetuneDataset.from_json(RETRIEVAL_EVAL_DATASET_FP)

2024-07-23 13:53:28.600 | INFO     | __main__:<module>:2 - Creating new synthetic retrieval eval dataset...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.60s/it]


### Evaluate

In [44]:
from llama_index.core.evaluation import RetrieverEvaluator

In [45]:
RETRIEVAL_METRICS = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    RETRIEVAL_METRICS, retriever=retriever
)

retrieval_eval_results = await retriever_evaluator.aevaluate_dataset(retrieval_eval_dataset)

> Top 2 nodes:
> [Node ed16c24c-937b-4bbb-80d7-dd33ea873087] [Similarity score:             0.699729] ) returns a document that describes Paul’s first programs: Node ID: 380fbb0e-6fc1-41de-a4f6-3f22c...
> [Node e358122f-b411-4c26-9694-f8dc08c13b2c] [Similarity score:             0.697677] GPT4-V Experiments with General, Specific questions and Chain Of Thought prompting(COT) technique...
> Top 2 nodes:
> [Node 486b3118-3b2d-455d-bcea-8dc8e0db26d2] [Similarity score:             0.749254] The retrieval approach entailed segmenting each document into 300-word sections, encoding both qu...
> [Node aaa56eee-5b1d-485a-b5f0-9905948c448a] [Similarity score:             0.735408] Generate Answers/Source Nodes (Context) Using List Index, we generate answers and source nodes fo...
> Top 2 nodes:
> [Node 3b5ad0bd-36ff-41f3-8197-81d880eeeb19] [Similarity score:             0.835285] Next, we can add the code that queries LlamaIndex: def   get_responses ( questions ):
    llm_ans...
> [Node 91ffc12

In [46]:
def display_results(name, eval_results, metrics=['hit_rate', 'mrr'], include_cohere_rerank=False):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    if include_cohere_rerank:
        crr_relevancy = full_df["cohere_rerank_relevancy"].mean()
        columns.update({"cohere_rerank_relevancy": [crr_relevancy]})

    metric_df = pd.DataFrame(columns)

    return metric_df

In [47]:
metric_prefix = f"top_{RETRIEVAL_TOP_K}_retrieval_eval"
retrieval_eval_results_df = display_results(metric_prefix, retrieval_eval_results, metrics=RETRIEVAL_METRICS)
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_2_retrieval_eval,0.1,0.1,0.05,0.1,0.1,0.061315


In [48]:
if LOG_TO_MLFLOW:
    for metric, metric_value in retrieval_eval_results_df.to_dict(orient='records')[0].items():
        if metric in RETRIEVAL_METRICS:
            mlflow.log_metric(f"{metric_prefix}_{metric}", metric_value)

### Manually curated dataset
Ref: https://docs.llamaindex.ai/en/stable/module_guides/evaluating/usage_pattern_retrieval/

In [49]:
MANUAL_EVAL_QA = [
("What are key features of llama-agents?",
"""
Key features of llama-agents are:
1. Distributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.
2. Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue.
3. Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task.
4. Ease of deployment: launch, scale and monitor each agent and your control plane independently.
5. Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent service
"""
),
("What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?",
"""
Retrieval System and Response Generation.
"""
),
("What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system?",
"""
Hit rate and Mean Reciprocal Rank (MRR)

Hit Rate: Hit rate calculates the fraction of queries where the correct answer is found within the top-k retrieved documents. In simpler terms, it’s about how often our system gets it right within the top few guesses.

Mean Reciprocal Rank (MRR): For each query, MRR evaluates the system’s accuracy by looking at the rank of the highest-placed relevant document. Specifically, it’s the average of the reciprocals of these ranks across all the queries. So, if the first relevant document is the top result, the reciprocal rank is 1; if it’s second, the reciprocal rank is 1/2, and so on.
"""
)
]

## Response Evaluation
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/downloading_llama_datasets/

In [101]:
def evaluate_labelled_rag_dataset(response_eval_dataset, response_eval_prediction_dataset, dataset_name="synthetic", batch_size=8, judge_model='gpt-3.5-turbo', cache_dp='.'):
    # Instantiate the judges
    judges = {
        "correctness": CorrectnessEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "relevancy": RelevancyEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "faithfulness": FaithfulnessEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "semantic_similarity": SemanticSimilarityEvaluator(),
    }

    # Initialize evaluations dictionary
    evals = {
        "correctness": [],
        "relevancy": [],
        "faithfulness": [],
        "contexts": [],
    }

    # Evaluate each prediction
    for example, prediction in tqdm(
        zip(response_eval_dataset.examples, response_eval_prediction_dataset.predictions)
    ):
        correctness_result = judges["correctness"].evaluate(
            query=example.query,
            response=prediction.response,
            reference=example.reference_answer,
        )

        relevancy_result = judges["relevancy"].evaluate(
            query=example.query,
            response=prediction.response,
            contexts=prediction.contexts,
        )

        faithfulness_result = judges["faithfulness"].evaluate(
            query=example.query,
            response=prediction.response,
            contexts=prediction.contexts,
        )

        evals["correctness"].append(correctness_result)
        evals["relevancy"].append(relevancy_result)
        evals["faithfulness"].append(faithfulness_result)
        evals["contexts"].append(prediction.contexts)

    # Save evaluations to JSON
    evaluations_objects = {
        "correctness": [e.dict() for e in evals["correctness"]],
        "faithfulness": [e.dict() for e in evals["faithfulness"]],
        "relevancy": [e.dict() for e in evals["relevancy"]],
        "contexts": evals['contexts'],
    }

    with open(f"{cache_dp}/{dataset_name}_evaluations.json", "w") as json_file:
        json.dump(evaluations_objects, json_file)

    # Generate evaluation results DataFrames
    deep_eval_correctness_df, mean_correctness_df = get_eval_results_df(
        ["base_rag"] * len(evals["correctness"]),
        evals["correctness"],
        metric="correctness",
    )
    deep_eval_relevancy_df, mean_relevancy_df = get_eval_results_df(
        ["base_rag"] * len(evals["relevancy"]),
        evals["relevancy"],
        metric="relevancy",
    )
    deep_eval_faithfulness_df, mean_faithfulness_df = get_eval_results_df(
        ["base_rag"] * len(evals["faithfulness"]),
        evals["faithfulness"],
        metric="faithfulness",
    )

    mean_scores_df = pd.concat(
        [
            mean_correctness_df.reset_index(),
            mean_relevancy_df.reset_index(),
            mean_faithfulness_df.reset_index(),
        ],
        axis=0,
        ignore_index=True,
    )
    mean_scores_df = mean_scores_df.set_index("index")
    mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])

    deep_eval_df = pd.concat([
        deep_eval_correctness_df[['query', 'answer']],
        deep_eval_relevancy_df[['scores']].rename(columns={'scores': 'relevancy_score'}),
        deep_eval_correctness_df[['scores']].rename(columns={'scores': 'correctness_score'}),
        deep_eval_faithfulness_df[['scores']].rename(columns={'scores': 'faithfulness_score'}),
        pd.Series(evals['contexts'], name='contexts')
    ], axis=1)

    return mean_scores_df, deep_eval_df

### Generate synthetic Llama Dataset

In [51]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.llama_dataset import LabeledRagDataset
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    SemanticSimilarityEvaluator,
)
from llama_index.core.evaluation.notebook_utils import get_eval_results_df
from llama_index.llms.openai import OpenAI

In [52]:
RECREATE_SYNTHETIC_EVAL_DATASET = True
RESPONSE_EVAL_DATASET_FP = f"{NOTEBOOK_CACHE_DP}/llamaindex_blog_response_eval_dataset.json"
RESPONSE_EVAL_LLM_MODEL = 'gpt-3.5-turbo'
RESPONSE_EVAL_LLM_MODEL_CONFIG = {
    "temperature": 0.3
}
SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK = 2
RESPONSE_NUM_SAMPLE_DOCUMENTS = 10
RESPONSE_NUM_SAMPLE_DOCUMENTS = min(len(documents), RESPONSE_NUM_SAMPLE_DOCUMENTS)

if LOG_TO_MLFLOW:
    mlflow.log_param("SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK", SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK)
    mlflow.log_param("RESPONSE_EVAL_LLM_MODEL", RESPONSE_EVAL_LLM_MODEL)
    mlflow.log_param("RESPONSE_NUM_SAMPLE_DOCUMENTS", RESPONSE_NUM_SAMPLE_DOCUMENTS)
    for k, v in RESPONSE_EVAL_LLM_MODEL_CONFIG.items():
        mlflow.log_param(f"RESPONSE_EVAL_LLM_MODEL_CONFIG__{k}", v)

In [53]:
if RESPONSE_NUM_SAMPLE_DOCUMENTS:
    logger.info(f"Sampling {RESPONSE_NUM_SAMPLE_DOCUMENTS} documents for response evaluation...")
    np.random.seed(41)
    response_eval_documents = np.random.choice(documents, RESPONSE_NUM_SAMPLE_DOCUMENTS)
else:
    logger.info(f"Using all documents for retrieval evaluation")
    response_eval_documents = documents

2024-07-23 13:54:08.602 | INFO     | __main__:<module>:2 - Sampling 10 documents for response evaluation...


In [54]:
if RECREATE_SYNTHETIC_EVAL_DATASET or not os.path.exists(RESPONSE_EVAL_DATASET_FP):
    logger.info(f"Creating synthetic response eval dataset...")
    # set context for llm provider
    response_eval_llm = OpenAI(model=RESPONSE_EVAL_LLM_MODEL, **RESPONSE_EVAL_LLM_MODEL_CONFIG)

    # instantiate a DatasetGenerator
    response_dataset_generator = RagDatasetGenerator.from_documents(
        response_eval_documents,
        llm=llm,
        num_questions_per_chunk=SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK,  # set the number of questions per nodes
        show_progress=True,
    )

    synthetic_response_eval_dataset = response_dataset_generator.generate_dataset_from_nodes()

    synthetic_response_eval_dataset.save_json(RESPONSE_EVAL_DATASET_FP)
else:
    logger.info(f"Loading existing synthetic response eval dataset at {RESPONSE_EVAL_DATASET_FP}...")
    synthetic_response_eval_dataset = LabeledRagDataset.from_json(RESPONSE_EVAL_DATASET_FP)

2024-07-23 13:54:10.804 | INFO     | __main__:<module>:2 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

> Adding chunk: LlamaIndex Newsletter 2024-03-05
Greetings, Lla...
> Adding chunk: OpenAI Cookbook: Evaluating RAG systems
We’re e...
> Adding chunk: LlamaIndex turns 1!
It’s our birthday! One year...
> Adding chunk: LlamaIndex Newsletter 2023–12–19
What’s up, Lla...
> Adding chunk: 👀 Community Demos : MemoryCache: Mozilla’s new ...
> Adding chunk: The latest updates to LlamaCloud
To build a pro...
> Adding chunk: Build and Evaluate LLM Apps with LlamaIndex and...
> Adding chunk: Finally, the third feedback function checks how...
> Adding chunk: Agentic RAG With LlamaIndex
The topic of Agenti...
> Adding chunk: These connectors can work with
  APIs, PDFs, SQ...
> Adding chunk: LlamaIndex Accelerates Enterprise Generative AI...
> Adding chunk: Introducing Llama Packs
Today we’re excited to ...
> Adding chunk: Special thanks to Logan Markewich and Andrei Fa...
> Adding chunk: LlamaIndex Newsletter 2023–11–14
Hello Llama Fr...
> Adding chunk: Finally, we released a guide to craft a GPT Bu

 40%|████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                          | 6/15 [00:07<00:10,  1.18s/it]

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.08747687122671666 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You have been rate limited. Your rate limit is 60 queries per minute. Please navigate to https://api.together.xyz/settings/billing to upgrade to a paid plan.', 'type': 'credit_limit', 'param': None, 'code': None}}.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.13s/it]


In [ ]:
synthetic_response_eval_prediction_dataset = await synthetic_response_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=8, show_progress=True
)

In [55]:
synthetic_mean_scores_df, synthetic_deep_eval_df = evaluate_labelled_rag_dataset(
    synthetic_response_eval_dataset,
    synthetic_response_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=RESPONSE_EVAL_LLM_MODEL,
    cache_dp=NOTEBOOK_CACHE_DP
)

Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/8 [00:00<?, ?it/s]

> Top 2 nodes:
> [Node e009b8ae-616f-47c3-9cf9-20fa1c683ac3] [Similarity score:             0.701982] OpenAI Cookbook: Evaluating RAG systems
We’re excited to unveil our  OpenAI Cookbook , a guide to...
> [Node ed16c24c-937b-4bbb-80d7-dd33ea873087] [Similarity score:             0.697224] ) returns a document that describes Paul’s first programs: Node ID: 380fbb0e-6fc1-41de-a4f6-3f22c...
> Top 2 nodes:
> [Node aaa56eee-5b1d-485a-b5f0-9905948c448a] [Similarity score:             0.757286] Generate Answers/Source Nodes (Context) Using List Index, we generate answers and source nodes fo...
> [Node eb2e69f6-6d0d-4810-bda8-b272aa270dc9] [Similarity score:             0.754412] Building and Evaluating a QA System with LlamaIndex
Introduction LlamaIndex (GPT Index)  offers a...
> Top 2 nodes:
> [Node 99874c4e-123d-4d24-b677-f4d581b37bd7] [Similarity score:             0.789413] LlamaIndex turns 1!
It’s our birthday! One year ago, Jerry pushed his  first commit  to GPT Index...
> [Node 0b40dc8

Batch processing of predictions:  50%|███████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                   | 4/8 [00:05<00:04,  1.21s/it]

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.8273668115595825 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You have been rate limited. Your rate limit is 60 queries per minute. Please navigate to https://api.together.xyz/settings/billing to upgrade to a paid plan.', 'type': 'credit_limit', 'param': None, 'code': None}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.6818801269681636 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You have been rate limited. Your rate limit is 60 queries per minute. Please navigate to https://api.together.xyz/settings/billing to upgrade to a paid plan.', 'type': 'credit_limit', 'param': None, 'code': None}}.


Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/8 [00:00<?, ?it/s]

...

Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/6 [00:00<?, ?it/s]

> Top 2 nodes:
> [Node 486b3118-3b2d-455d-bcea-8dc8e0db26d2] [Similarity score:             0.783261] The retrieval approach entailed segmenting each document into 300-word sections, encoding both qu...
> [Node e149477d-6dc2-4238-9b63-18878e66cc45] [Similarity score:             0.759313] One is indexing document summaries and linking them to documents, and the other is indexing small...
> Top 2 nodes:
> [Node 7f51966e-f508-4e4d-9103-b9ae9d81d6c3] [Similarity score:             0.804341] We also offer an extensive array of integrations with other storage providers and downstream appl...
> [Node e70db3b1-e06a-4d17-b88d-6da062fa67ca] [Similarity score:             0.794489] Building the data framework for LLMs
Today is an exciting day for LlamaIndex, and a big milestone...
> Top 2 nodes:
> [Node aaa56eee-5b1d-485a-b5f0-9905948c448a] [Similarity score:             0.756688] Generate Answers/Source Nodes (Context) Using List Index, we generate answers and source nodes fo...
> [Node eb2e69f

Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.56s/it]


0it [00:00, ?it/s]

> Adding chunk: Generate Answers/Source Nodes (Context) Using L...
> Adding chunk: Building and Evaluating a QA System with LlamaI...
> Adding chunk: Generate Answers/Source Nodes (Context) Using L...
> Adding chunk: Building and Evaluating a QA System with LlamaI...
> Adding chunk: LlamaIndex newsletter 2023–10–24
Hello Llama Fa...
> Adding chunk: Notebook ,  Tweet . RAGatouille LlamaPack : Int...
> Adding chunk: LlamaIndex newsletter 2023–10–24
Hello Llama Fa...
> Adding chunk: Notebook ,  Tweet . RAGatouille LlamaPack : Int...
> Adding chunk: The retrieval approach entailed segmenting each...
> Adding chunk: One is indexing document summaries and linking ...
> Adding chunk: The retrieval approach entailed segmenting each...
> Adding chunk: One is indexing document summaries and linking ...
> Adding chunk: OpenAI Cookbook: Evaluating RAG systems
We’re e...
> Adding chunk: ) returns a document that describes Paul’s firs...
> Adding chunk: OpenAI Cookbook: Evaluating RAG systems
We’re 

In [56]:
synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.616667
mean_relevancy_score,0.766667
mean_faithfulness_score,0.666667


In [57]:
synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score
0,"Based on the context information, I've generat...","I'd be happy to help! Based on the context, I ...",1.0,3.5,1.0
1,**Question 1:** What is the new feature releas...,There is no new feature released in LlamaIndex...,0.0,2.0,1.0
2,Here are two questions based on the context in...,Here are the answers to the two questions base...,1.0,3.0,1.0
3,**Question 1:** What is the primary goal of th...,To provide the community with an essential res...,0.0,3.5,1.0
4,"Based on the context information, I've generat...",I'd be happy to help!,1.0,3.0,0.0
5,**Question 1:** What is the primary limitation...,"The primary limitation of GPT-3, according to ...",1.0,4.0,1.0
6,"Based on the context information, I've generat...",I'd be happy to help!,1.0,1.0,0.0
7,**Question 1:** What is the name of the new da...,The new dataset launched by LlamaIndex in coll...,0.0,4.5,1.0
8,"Based on the context information, I've generat...",I'd be happy to help!,1.0,1.0,0.0
9,**Question 1:** What is the name of the experi...,There is no information in the provided contex...,0.0,1.0,1.0


In [58]:
if LOG_TO_MLFLOW:
    for k, v in synthetic_mean_scores_df.T.to_dict(orient='records')[0].items():
        mlflow.log_metric(f"synthetic_response_eval__{k}", v)
    synthetic_deep_eval_df.to_html(f"{NOTEBOOK_CACHE_DP}/synthetic_deep_eval_df.html")
    mlflow.log_artifact(f"{NOTEBOOK_CACHE_DP}/synthetic_deep_eval_df.html", "synthetic_deep_eval_df")

### Manually curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [59]:
from llama_index.core.llama_dataset import LabelledRagDataset, LabelledRagDataExample, CreatedBy, CreatedByType

examples = []

for question, expected_anwser in MANUAL_EVAL_QA:
    example = LabelledRagDataExample(
        query=question,
        query_by=CreatedBy(type=CreatedByType.HUMAN),
        reference_answer=expected_anwser,
        reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),
        reference_contexts=[],
    )
    examples.append(example)

curated_response_eval_dataset = LabelledRagDataset(examples=examples)

# save this dataset as it is required for the submission
curated_response_eval_dataset.save_json(f"{NOTEBOOK_CACHE_DP}/curated_response_eval_dataset.json")

In [72]:
curated_response_eval_prediction_dataset = await curated_response_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=8, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/3 [00:00<?, ?it/s]

> Top 2 nodes:
> [Node e009b8ae-616f-47c3-9cf9-20fa1c683ac3] [Similarity score:             0.855356] OpenAI Cookbook: Evaluating RAG systems
We’re excited to unveil our  OpenAI Cookbook , a guide to...
> [Node 0020f723-6a45-4396-9543-f414375e8936] [Similarity score:             0.781137] LlamaIndex Newsletter 2023–12–05
Hello Llama Community 🦙, We are excited to collaborate with Deep...
> Top 2 nodes:
> [Node 685a355d-9332-41a9-96bd-a4903ec86777] [Similarity score:             0.824755] Implemented by the user.

        """ 
         return  self._retrieve(query_bundle)

     async ...
> [Node d175dd37-b2cb-4772-9709-4cc90227bac0] [Similarity score:             0.81401] # Extract keys from queries and relevant_docs that need to be removed 
    queries_relevant_docs_...
> Top 2 nodes:
> [Node 78ead847-3d01-4db3-8483-59a88f95b1d7] [Similarity score:             0.797389] First we’ll bring in our dependencies and set up our control plane, which contains our LLM-powere...
> [Node 9929ccb4

Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.88s/it]


In [102]:
curated_mean_scores_df, curated_deep_eval_df = evaluate_labelled_rag_dataset(
    curated_response_eval_dataset,
    curated_response_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=RESPONSE_EVAL_LLM_MODEL,
    cache_dp=NOTEBOOK_CACHE_DP
)

0it [00:00, ?it/s]

> Adding chunk: First we’ll bring in our dependencies and set u...
> Adding chunk: Introducing llama-agents: A Powerful Framework ...
> Adding chunk: First we’ll bring in our dependencies and set u...
> Adding chunk: Introducing llama-agents: A Powerful Framework ...
> Adding chunk: OpenAI Cookbook: Evaluating RAG systems
We’re e...
> Adding chunk: LlamaIndex Newsletter 2023–12–05
Hello Llama Co...
> Adding chunk: OpenAI Cookbook: Evaluating RAG systems
We’re e...
> Adding chunk: LlamaIndex Newsletter 2023–12–05
Hello Llama Co...
> Adding chunk: Implemented by the user.

        """ 
        ...
> Adding chunk: # Extract keys from queries and relevant_docs t...
> Adding chunk: Implemented by the user.

        """ 
        ...
> Adding chunk: # Extract keys from queries and relevant_docs t...


In [103]:
curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.5
mean_relevancy_score,1.0
mean_faithfulness_score,1.0


In [104]:
with pd.option_context('display.max_colwidth', None):
    display(curated_deep_eval_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are key features of llama-agents?,"Distributed Service-Oriented Architecture, Communication via standardized API interfaces, Pass messages between agents using a message queue, Define agentic and explicit orchestration flows, Ease of deployment, Scalability and resource management.",1.0,NaN,1.0,"[First we’ll bring in our dependencies and set up our control plane, which contains our LLM-powered orchestrator import dotenv\ndotenv.load_dotenv() # our .env file defines OPENAI_API_KEY \n from llama_agents import (\n AgentService,\n ControlPlaneServer,\n SimpleMessageQueue,\n AgentOrchestrator,\n)\n from llama_index.core.agent import FunctionCallingAgentWorker\n from llama_index.core.tools import FunctionTool\n from llama_index.llms.openai import OpenAI\n import logging\n\n # turn on logging so we can see the system working \nlogging.getLogger( ""llama_agents"" ).setLevel(logging.INFO)\n\n # Set up the message queue and control plane \nmessage_queue = SimpleMessageQueue()\ncontrol_plane = ControlPlaneServer(\n message_queue=message_queue,\n orchestrator=AgentOrchestrator(llm=OpenAI()),\n) Next we create our tools using LlamaIndex’s existing abstractions, provide those tools to an agent, and turn that agent into an independent microservice: # create a tool \n def get_the_secret_fact () -> str :\n """"""Returns the secret fact."""""" \n return ""The secret fact is: A baby llama is called a 'Cria'."" \n\ntool = FunctionTool.from_defaults(fn=get_the_secret_fact)\n\n # Define an agent \nworker = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI())\nagent = worker.as_agent()\n\n # Create an agent service \nagent_service = AgentService(\n agent=agent,\n message_queue=message_queue,\n description= ""General purpose assistant"" ,\n service_name= ""assistant"" ,\n) Finally we launch the service and the control plane. Note that here we’re using a helper function to run a single query through the system and then exit; next we’ll show how to deploy this to production. # Set up the launcher for local testing \n from llama_agents import LocalLauncher\n\nlauncher = LocalLauncher(\n [agent_service],\n control_plane,\n message_queue,\n)\n\n # Run a single query through the system \nresult = launcher.launch_single( ""What's the secret fact?"", Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems\nWe're excited to announce the alpha release of llama-agents , a new open-source framework designed to simplify the process of building, iterating, and deploying multi-agent AI systems and turn your agents into production microservices. Whether you're working on complex question-answering systems, collaborative AI assistants, or distributed AI workflows, llama-agents provides the tools and structure you need to bring your ideas to life. Key Features of llama-agents Distributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks. Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue. Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task. Ease of deployment: launch, scale and monitor each agent and your control plane independently. Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent service Let's dive into how you can start using llama-agents to build your own multi-agent systems. Getting Started with llama-agents First, install the framework using pip: pip install llama-agents llama-index-a

In [109]:
for context in curated_deep_eval_df.iloc[2]['contexts']:
    print(context)
    print('-' * 10)

Implemented by the user.

        """ 
         return  self._retrieve(query_bundle)

     async   def   aretrieve ( self, str_or_query_bundle: QueryType ) -&gt;  List [NodeWithScore]:
         if   isinstance (str_or_query_bundle,  str ):
            str_or_query_bundle = QueryBundle(str_or_query_bundle)
         return   await  self._aretrieve(str_or_query_bundle)

custom_retriever = CustomRetriever(vector_retriever) Evaluation: To evaluate our retriever, we computed the Mean Reciprocal Rank (MRR) and Hit Rate metrics: retriever_evaluator = RetrieverEvaluator.from_metric_names(
    [ "mrr" ,  "hit_rate" ], retriever=custom_retriever
)
eval_results =  await  retriever_evaluator.aevaluate_dataset(qa_dataset) Results: We put various embedding models and rerankers to the test. Here are the models we considered: Embedding Models : OpenAI Embedding Voyage Embedding CohereAI Embedding  (v2.0/ v3.0) Jina Embeddings  (small/ base) BAAI/bge-large-en Google PaLM Embedding Rerankers : CohereAI b

In [67]:
if LOG_TO_MLFLOW:
    for k, v in curated_mean_scores_df.T.to_dict(orient='records')[0].items():
        mlflow.log_metric(f"curated_response_eval__{k}", v)
    curated_deep_eval_df.to_html(f"{NOTEBOOK_CACHE_DP}/curated_deep_eval_df.html")
    mlflow.log_artifact(f"{NOTEBOOK_CACHE_DP}/curated_deep_eval_df.html", "curated_deep_eval_df")

# Clean up

In [68]:
if LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive